# Exercise - approximating a stress-strain relation with a neural network

In this exercise, you will build a neural network to reconstruct the model parameters from given stress-strain curves. Let us assume that the stress-strain relation is well approximated [_Ramberg-Osgood_](https://en.wikipedia.org/wiki/Ramberg–Osgood_relationship) model, which represents a non-linear relation between the stress $\sigma$  and the strain $\epsilon$, namely
$$ \sigma = C \epsilon^{1/n}.$$
Here, $C$ and $n$ denote the material parameters, which should be identified based on given stress-strain curves. Specifically, for each stress-strain curve, the model should output a pair $(C,n)$: hence, we are dealing with a _multi-target regression_ problem.

In [20]:
import matplotlib.pyplot as plt
from data.data_generator import StressStrainDataGenerator
from torch import nn
import torch
from skorch import NeuralNetRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split

In [3]:
# Auxiliary functions
def get_stress_from_model(eps, C, n):
    return C*eps**(1.0/n)

In [4]:
# generate dataset: each sample (row) consists of strains and stresses arranged sequentially 
# (first NUM_DATAPOINTS columns corresponds to strains, the rest to the associated stresses)
NUM_DATAPOINTS = 101
stressStrainDataGenerator = StressStrainDataGenerator(num_datapoints=NUM_DATAPOINTS) 
stressStrainDataGenerator.std_noise =  0.554
X, y = stressStrainDataGenerator.generate_stress_strain_datasets(num_samples=2000,
                                                                          C_range=[5,15],
                                                                          n_range=[2,5],
                                                                          shuffle=True,
                                                                          add_noise=True,
                                                                          eps_linear_distributed=False)

1. Plot 6 stress-strain curves of the dataset using a scatter plot. Display a legend that relates the values of $C$ and $n$ to each curve.

2. Split the dataset into training and test sets (0.8-0.2) using the _scikit-learn_ function [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). Call the resulting arrays `X_train`, `X_test`, `y_train`, `y_test`.

3. Create a _PyTorch_ neural network class that implements a multi-layer perceptron to predict the values of $C$ and $n$ given an array combining strains and stresses. Fix the number of hidden layers to 3 and define a parameter called `hidden_units` in the constructor of the class that allows to change the number of units for each hidden layer (for simplicity, assume that the number is the same for all hidden layers).

**NOTE**: in the `forward` method of the class, process the input _x_ with the instruction `x = x.to(torch.float32)` before passing it to the hidden layers.

In [55]:
# these tensors must be used for the training of the neural network
X_ = torch.tensor(X_train, dtype=torch.float32)
y_ = torch.tensor(y_train, dtype=torch.float32)

In [60]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # Define the layers here
        # ...

    def forward(self, x):
        x = x.to(torch.float32)
        # ...
        return x

4. Create a _skorch_ `NeuralNetRegressor` object (see the docs [here](https://skorch.readthedocs.io/en/stable/regressor.html)) that wraps the Pytorch model. Set the learning rate equal to 0.01 and the batch size equal to 64. Disable train/validation split, set the parameter `verbose=0` (to avoid printing anything during the grid search runs, see point 6) and set the number of epochs equal to 20. Check that the optimizer is set to SGD and that the loss function is MSE. When calling the _NeuralNetRegressor_ constructor, also pass the parameter `module__hidden_units=10` to set the default number of hidden units.   

In [58]:
# model = NeuralNetRegressor(...)

5. Create a _scikit-learn `Pipeline` object that appropriately standardizes the dataset and feeds it to the _NeuralNetRegressor_ object. Name the model object in the pipeline as `model`. 

In [57]:
pipe = Pipeline([
# ...
# ...
])

6. Use the [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) interface of _scikit-learn_ to carry out a grid search on the parameter called `model__module__hidden_units` (which is passed to the `model` in the pipeline that you just built). Use 3-fold cross-validation. Also check the _User Guide_ of _scikit-learn_ about the grid search. Print the number of hidden units that results in the best validation score.

7. Plot the training loss vs number of epochs for the best model found in the grid search.

8. Using the best model obtained from the grid search, generate the predictions over the test set and evaluate the $R^2$ on this set. Plot the stresses and the strains corresponding to one sample of the test set along with the curve obtained using the function `get_stress_from_model` with the values of $C$ and $n$ estimated by the trained neural network.